In [1]:
import sys
sys.path.append('../')

import networkx as nx
from node2vec import Node2Vec

from src.utils import load_embeddings

%load_ext autoreload
%autoreload 2

In [2]:
G = nx.read_gpickle('data/czech_road_network.p')
G.number_of_nodes(), G.number_of_edges()

(1128, 1153)

In [3]:
def generate_node_embeddings(graph: nx.Graph):
    model = Node2Vec(graph, dimensions=32, num_walks=100, walk_length=80, weight_key='dist', temp_folder='temp/', workers=2)
    result = model.fit(window=5, min_count=1)
    return result

In [5]:
%%time
embeddings = generate_node_embeddings(G)

Generating walks (CPU: 1): 100%|██████████| 50/50 [06:10<00:00,  7.41s/it]


CPU times: user 7min 6s, sys: 53 s, total: 7min 59s
Wall time: 6min 34s


In [6]:
for idx, e in enumerate(G.edges()):
    if idx == 5:
        break
    print(e)

('D00-001', 'D00-002')
('D00-001', 'D10-001')
('D00-002', 'D00-003')
('D00-002', 'D11-001')
('D00-003', 'D00-004')


In [7]:
embeddings.wv.most_similar('D00-001')

[('D10-001', 0.935806393623352),
 ('D00-002', 0.8871928453445435),
 ('D10-002', 0.7911334037780762),
 ('D00-003', 0.6844806671142578),
 ('D11-001', 0.683589518070221),
 ('D11-002', 0.6108906269073486),
 ('D10-003', 0.5654127597808838),
 ('I12-001', 0.4950152039527893),
 ('I12-002', 0.46955227851867676),
 ('D11-003', 0.41467419266700745)]

In [9]:
embeddings.save('data/embeddings32.data')

In [ ]:
# join embeddings with corresponding nodes (as x feature)
for idx, node in enumerate(G.nodes()):
    G.add_node(node, embedding=embeddings.wv[str(node)].copy())

In [ ]:
nx.readwrite.write_gpickle(G, 'data/network32.gpickle')